In [1]:
import librosa
import sys
import pathlib

ROOT = pathlib.Path(".").absolute().parent
sys.path.append(str(ROOT))


In [23]:
from utils.metrics import (
    plot_classification_report,
    print_scores
)
import os
import pickle
from datasets.SPCUP22DataModule import SPCUP22DataModule
from features.audio import MFCC
from utils.config import load_config_file
from torchvision.transforms import Compose
from tqdm import tqdm
import numpy as np

import numpy as np
from sklearn.mixture import GaussianMixture

In [4]:
# Batch size and other constant
batch_size = 128

In [6]:
# Download and load the data
data_module = SPCUP22DataModule(
    batch_size=batch_size,
    dataset_root=str(ROOT.joinpath("data", "spcup22").absolute()),
    
)
data_module.prepare_data()
data_module.setup()
train_loader = data_module.train_dataloader()
test_dataloader = data_module.test_dataloader()

Skipping downloading [/u/20/hassans4/data/Documents/postdoctoral/colab/spcup22/repository/SPCUP2022/data/spcup22/spcup_2022_training_part1.zip]...
Skipping downloading [/u/20/hassans4/data/Documents/postdoctoral/colab/spcup22/repository/SPCUP2022/data/spcup22/spcup_2022_unseen.zip]...
Skipping downloading [/u/20/hassans4/data/Documents/postdoctoral/colab/spcup22/repository/SPCUP2022/data/spcup22/spcup_2022_eval_part1.zip]...


In [10]:
len(train_loader.dataset)

3600

In [19]:
# Load data in batch and convert to features
nsamples = len(train_loader.dataset)
nfeatures = 188
train_labels = np.zeros((nsamples,))
train_features =  np.zeros((nsamples, nfeatures))
sample_index = 0

for batch in train_loader:
    batch_samples, batch_labels = batch
    ys = batch_samples.cpu().detach().numpy()
    labels = batch_labels.cpu().detach().numpy()
    
    for idx, y in enumerate(ys):
        feature = librosa.feature.spectral_centroid(y=y, sr=22050)
        feature = np.squeeze(feature)
        
        train_labels[sample_index] = labels[idx]
        train_features[sample_index, :] = feature
        sample_index = sample_index + 1
    
    
    


In [24]:
gm = GaussianMixture(n_components=6, random_state=0).fit(train_features)

In [26]:
train_labels[0:10]

array([3., 0., 1., 0., 4., 3., 1., 0., 3., 2.])

In [25]:
gm.predict(train_features[0:10, : ])

array([5, 1, 4, 2, 1, 3, 0, 1, 5, 5])

In [27]:
gm.predict_proba(train_features[0:10, : ])

array([[2.69452782e-043, 1.75402438e-031, 3.95125867e-089,
        1.49740514e-044, 1.35373142e-030, 1.00000000e+000],
       [1.42659146e-037, 1.00000000e+000, 8.25606368e-075,
        2.76978741e-034, 3.58225337e-059, 2.21478451e-047],
       [2.23676889e-037, 1.33574786e-019, 9.85856311e-072,
        2.30033069e-045, 1.00000000e+000, 1.98851066e-046],
       [4.04848333e-063, 4.87791239e-031, 1.00000000e+000,
        1.51423004e-060, 2.33765700e-100, 5.75762456e-094],
       [3.65984757e-013, 9.99877031e-001, 1.22969222e-004,
        2.17232166e-010, 6.11462775e-014, 2.35119024e-019],
       [2.65462343e-028, 1.26147056e-022, 3.01845915e-084,
        1.00000000e+000, 4.48898854e-056, 4.55935158e-071],
       [1.00000000e+000, 5.50196494e-017, 1.14395997e-060,
        7.83997177e-023, 9.65077231e-024, 7.65334363e-036],
       [1.14029538e-027, 1.00000000e+000, 1.09011271e-069,
        4.89669867e-011, 3.92484513e-022, 1.17077825e-035],
       [2.77463816e-021, 1.74491119e-016, 1.1251